In [1]:
from pathlib import Path
import sys

sys.path.append("..")
import constants

SURVEY_DIR = Path("/media/ko/MAUI63/2025-11-21-rotorua-test-flight/")
OUTPUT_DIR = constants.ROOT_DIR / "survey-review-app" / "data" / "2025-11-21-rotorua-test-flight"
ABOVE_SEA_LEVEL_METERS = 940 / 3.333  # Rotorua at 940ft

print(f"Using survey dir: {SURVEY_DIR}")
print(f"Using output dir: {OUTPUT_DIR}")
print(f"Using above sea level offset: {ABOVE_SEA_LEVEL_METERS} meters")

Using survey dir: /media/ko/MAUI63/2025-11-21-rotorua-test-flight
Using output dir: /home/ko/customers/maui63/survey-computer-vision/survey-review-app/data/2025-11-21-rotorua-test-flight
Using above sea level offset: 282.028202820282 meters


In [2]:
# Get all the frames:
from collections import namedtuple
from datetime import datetime, timedelta
import pytz

GpsTriggerEvent = namedtuple("GpsTriggerEvent", ["time", "lat", "lon", "alt"])

# Parse the GPS data:
gps_points = []
for fpath in sorted(SURVEY_DIR.rglob("*.CAM")):
    # Parse the start time from the filename:
    # Read the files:
    with open(fpath, "r") as f:
        for line_idx, line in enumerate(f):
            parts = line.split(",")
            week = int(parts[0])
            start_time = datetime(1980, 1, 6, tzinfo=pytz.utc) + timedelta(weeks=week)
            milliseconds_since_start_of_week = float(parts[1])
            num_satellites = int(parts[7])
            lat = float(parts[8])
            lon = float(parts[9])
            alt = float(parts[10]) - ABOVE_SEA_LEVEL_METERS  # Convert to above ground level
            time = start_time + timedelta(milliseconds=milliseconds_since_start_of_week)
            gps_points.append(GpsTriggerEvent(time.isoformat(), lat, lon, alt))
print(f"Parsed {len(gps_points)} GPS points")

Parsed 1562 GPS points


In [ ]:
from collections import Counter

altitudes = [int(gp.alt / 10) * 10 for gp in gps_points]
altitude_counts = Counter(altitudes)
print("Altitude distribution (meters above ground level):")
for alt, count in sorted(altitude_counts.items()):
    print(f"  {alt} m: {count} images")

Altitude distribution (meters above ground level):
  10 m: 1 images
  20 m: 3 images
  160 m: 21 images
  170 m: 164 images
  180 m: 171 images
  190 m: 77 images
  200 m: 25 images
  210 m: 10 images
  220 m: 3 images
  230 m: 3 images
  240 m: 4 images
  250 m: 4 images
  260 m: 4 images
  270 m: 2 images
  280 m: 3 images
  290 m: 14 images
  300 m: 4 images
  310 m: 3 images
  320 m: 4 images
  330 m: 7 images
  340 m: 68 images
  350 m: 177 images
  360 m: 76 images
  370 m: 37 images
  380 m: 26 images
  390 m: 20 images
  400 m: 1 images
  410 m: 3 images
  420 m: 2 images
  430 m: 2 images
  440 m: 2 images
  450 m: 2 images
  460 m: 2 images
  470 m: 2 images
  480 m: 1 images
  490 m: 2 images
  500 m: 2 images
  510 m: 2 images
  520 m: 2 images
  530 m: 2 images
  790 m: 13 images
  800 m: 288 images
  810 m: 286 images
  820 m: 17 images


In [4]:
gps_points[0]

GpsTriggerEvent(time='2025-11-20T23:57:04.783000+00:00', lat=-38.108082056, lon=176.318081566, alt=25.484997179717993)

In [5]:
# Get all the photos:
import json
from img_utils import get_exif_lazy
from tqdm import tqdm


camera_photos = {}
processed_frames = []
possible_detections = []
for daydir in sorted(SURVEY_DIR.iterdir()):
    if not daydir.is_dir():
        continue

    # Load GPS matched stuff:
    print("Reading ", daydir / "frames.json")
    with open(daydir / "frames.json") as f:
        frames = json.load(f)["frames"]
    # Add full path
    for f in frames:
        for i in f["images"].values():
            exifimg = get_exif_lazy(daydir / i["path"])
            i["path"] = f"{daydir.name}/{i['path']}"
            # Add aperture, shutter speed, ISO
            i["aperture"] = exifimg["f_number"]
            i["shutter_speed"] = exifimg["exposure_time"]
            i["iso"] = exifimg["photographic_sensitivity"]
    processed_frames.extend(frames)

    # Load possible detections
    inference_dir = daydir / "inferences"
    if inference_dir.exists():
        for camera_dir in inference_dir.iterdir():
            for reviewed_img in (camera_dir / "reviewed").rglob("*.jpg"):
                relpath = reviewed_img.relative_to(camera_dir / "reviewed")
                inferences_path = camera_dir / "inferences" / relpath.with_suffix(".json")
                with open(inferences_path) as f:
                    detections = json.load(f)
                assert len(detections) > 0
                img_path = daydir / "cameras" / camera_dir.name / "DCIM" / relpath.with_suffix(".JPG")
                assert img_path.exists(), f"Image path {img_path} does not exist"
                possible_detections.append(
                    {
                        "camera": camera_dir.name,
                        "img_path": img_path,
                        "detections": detections,
                    }
                )

    # Load frames
    cameras_dir = daydir / "cameras"
    for camera_dir in sorted(cameras_dir.iterdir()):
        camera_name = camera_dir.name
        print(f"Getting photos from {camera_dir}")
        for img_path in sorted(camera_dir.rglob("*.JPG")):
            if camera_name not in camera_photos:
                camera_photos[camera_name] = []
            camera_photos[camera_name].append(img_path)

Reading  /media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/frames.json
Getting photos from /media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/l09
Getting photos from /media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/l28
Getting photos from /media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/r09
Getting photos from /media/ko/MAUI63/2025-11-21-rotorua-test-flight/2025-11-21/cameras/r28


In [6]:
print(f"Loaded {len(processed_frames)} gps points")
print(f"Loaded {sum(len(v) for v in camera_photos.values())} photos")
print(f"Loaded {len(possible_detections)} possible detections")

Loaded 1562 gps points
Loaded 3148 photos
Loaded 0 possible detections


In [7]:
# OK, for each gps point, get the bounding box of each image so we can show it on a map
import pyproj

geodesic = pyproj.Geod(ellps="WGS84")

# TODO: calculate offsets from altitude
offsets = {
    "l28": (-550, -250),
    "l09": (-250, 0),
    "r09": (0, 250),
    "r28": (250, 550),
}
image_height = 200
sorted_frames = sorted(processed_frames, key=lambda x: x["gps_time"])
for idx in range(len(sorted_frames)):
    p1 = sorted_frames[idx]
    bearing = 0
    if idx > 0:
        p0 = sorted_frames[idx - 1]
        bearing, _, _ = geodesic.inv(p0["lon"], p0["lat"], p1["lon"], p1["lat"])

    for cam, d in p1["images"].items():
        x0, x1 = offsets[cam]
        forward_lon, forward_lat, _ = geodesic.fwd(p1["lon"], p1["lat"], bearing, image_height / 2)
        rear_lon, rear_lat, _ = geodesic.fwd(p1["lon"], p1["lat"], bearing + 180, image_height / 2)
        angle = bearing - 90 if x0 < 0 else bearing + 90
        top_left_lon, top_left_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x0))
        top_right_lon, top_right_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x1))
        bottom_left_lon, bottom_left_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x0))
        bottom_right_lon, bottom_right_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x1))

        d["bounds_lbrt"] = [
            [bottom_left_lon, bottom_left_lat],
            [top_left_lon, top_left_lat],
            [top_right_lon, top_right_lat],
            [bottom_right_lon, bottom_right_lat],
        ]

processed_frames[0]

{'lat': -38.108082056,
 'lon': 176.318081566,
 'agl': 25.484997179717993,
 'gps_time': '2025-11-20T23:57:04.783000+00:00',
 'images': {'l09': {'path': '2025-11-21/cameras/l09/10251121/_09L9114.JPG',
   't': '2025-11-21T12:57:04.638000+13:00',
   'aperture': 4.0,
   'shutter_speed': 0.0004,
   'iso': 2500,
   'bounds_lbrt': [[176.31523101422184, -38.108982934242654],
    [176.31523108423875, -38.10718110846564],
    [176.318081566, -38.107181143041124],
    [176.318081566, -38.10898296882037]]},
  'l28': {'path': '2025-11-21/cameras/l28/10251121/_28L9119.JPG',
   't': '2025-11-21T12:57:04.678000+13:00',
   'aperture': 2.8,
   'shutter_speed': 0.0004,
   'iso': 1600,
   'bounds_lbrt': [[176.3118103520956, -38.10898280146423],
    [176.31181050613284, -38.107180975695755],
    [176.31523108423875, -38.10718110846564],
    [176.31523101422184, -38.108982934242654]]},
  'r09': {'path': '2025-11-21/cameras/r09/10151121/_09R5872.JPG',
   't': '2025-11-21T12:57:04.639000+13:00',
   'aperture':

In [8]:
from PIL import Image

# Convert detection sizes to x,y,w,h relative:
for det in possible_detections:
    path = det["img_path"]
    with Image.open(path) as img:
        iw, ih = img.size
    for d in det["detections"]:
        x0, y0, x1, y1 = d["x0"], d["y0"], d["x1"], d["y1"]
        left = x0 / iw
        top = y0 / ih
        width = (x1 - x0) / iw
        height = (y1 - y0) / ih
        d["ltwh"] = [left, top, width, height]
possible_detections[0] if possible_detections else None

In [9]:
import json

# Save it out:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
with open(OUTPUT_DIR / "frames.json", "w") as f:
    json.dump(processed_frames, f, indent=2)
with open(OUTPUT_DIR / "camera_photos.json", "w") as f:
    json.dump({k: [str(p.relative_to(SURVEY_DIR)) for p in sorted(v)] for k, v in camera_photos.items()}, f, indent=2)
with open(OUTPUT_DIR / "possible_detections.json", "w") as f:
    json.dump(
        [
            {
                "camera": d["camera"],
                "img_path": str(d["img_path"].relative_to(SURVEY_DIR)),
                "detections": d["detections"],
            }
            for d in possible_detections
        ],
        f,
        indent=2,
    )
# Get the first image from camera l28
